# 首先是資料整理的部分，x_list 裡存的是轉成數字陣列的圖片，y_list 裡存的是類別，這裡用 0 代表貓，1代表狗。

In [1]:
import os # 用來處理各種系統上的事情的套件
from PIL import Image # 讀圖片用的套件
import numpy # 處理數字資料的套件
import keras # 比較簡單的 deep learning 套件
from sklearn.utils import shuffle # 資料分析常用的套件，這次只用它裡面的 shuffle 功能
x_train = [] # 用來存轉成數字陣列的圖片，也就是你要輸進模型的 feature
y_train = [] # 用來存我們要的 target，0 代表貓，1 代表狗
for image_name in os.listdir("train"): # os.listdir 可以列出資料夾裡的所有檔案名稱，括號裡放 train 代表我要看 train 資料夾裡的所有檔案
    img = Image.open("train/" + image_name) # Image.open 可以開啟圖片檔案，不過他還不是陣列，所以我們還要再做轉換
    img = img.resize((224,224)) # 把圖片統一轉成 224 * 224
    img = numpy.array(img) # 把剛剛開啟的圖片放到 numpy.array 裡就可以轉成陣列
    x_train.append(img) # 把轉成數字陣列的圖片放到 x_list 裡
    if("cat" in image_name): # 如果檔案名稱裡有 cat 的話，把 0(貓) 放到 y_list 裡，否則放 1(狗) 到 y_list 裡
        y_train.append(0)
    else:
        y_train.append(1)

x_train = numpy.array(x_train) # 模型只吃 numpy.array 型態的資料，所以先把 x_train 轉成 numpy.array
x_train = x_train / 255 # 把所有 pixel 的值標準化到 0 到 1 之間
y_train = keras.utils.to_categorical(y_train, num_classes=2) # 把 y_train 做 one hot encoding 
x_train, y_train = shuffle(x_train, y_train) # 用 shuffle 可以打亂資料，並且不會改變 x_train 對應的 y_train

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 再來訓練資料的輸入和輸出都準備好了，就可以建一個模型開始訓練了。
https://keras.io/getting-started/sequential-model-guide/

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3))) # input_shape 改成你圖片的 size 
model.add(Conv2D(32, (3, 3), activation='relu')) # 以這行來講，它有 32 個 filter，每個 filter 的 size 是 3*3
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) # 把 CNN 抽出來的 feature map 攤平，這樣才能餵進 Dense 做分類
model.add(Dense(256, activation='relu'))
model.add(Dense(15, activation='softmax')) # 分 2 類所以神經元數量設 2，做分類所以 activation 用 softmax

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics = ["acc"])# 做分類所以 loss 用 categorical_crossentropy

model.fit(x_train, y_train, batch_size=64, epochs=5) # batch_size 不一定多少，建議用 32

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/5
25000/25000 [==============================] - 128s 5ms/step - loss: 0.6930 - acc: 0.5370
Epoch 2/5
25000/25000 [==============================] - 81s 3ms/step - loss: 0.6360 - acc: 0.6366
Epoch 3/5
25000/25000 [==============================] - 80s 3ms/step - loss: 0.5071 - acc: 0.7544
Epoch 4/5
25000/25000 [==============================] - 80s 3ms/step - loss: 0.4001 - acc: 0.8200
Epoch 5/5
25000/25000 [==============================] - 80s 3ms/step - loss: 0.2737 - acc: 0.8857


# 模型訓練好就可以把 x_test 丟進模型，再把輸出結果整理成 kaggle 上要求的格式，就可以上傳了。

In [3]:
import pandas
x_test = [] # 用來存轉成數字陣列的圖片，也就是你要輸進模型的 feature
ids = []
for image_name in os.listdir("test1"): # os.listdir 可以列出資料夾裡的所有檔案名稱，括號裡放 test1 代表我要看 test1 資料夾裡的所有檔案
    img = Image.open("test1/" + image_name) # Image.open 可以開啟圖片檔案，不過他還不是陣列，所以我們還要再做轉換
    img = img.resize((224,224)) # 把圖片統一轉成 224 * 224
    img = numpy.array(img) # 把剛剛開啟的圖片放到 numpy.array 裡就可以轉成陣列
    ids.append(image_name.split(".jpg")[0]) # 把檔名前的 id 抓出來
    x_test.append(img)
x_test = numpy.array(x_test) # 模型只吃 numpy.array 型態的資料，所以先把 x_test 轉成 numpy.array
x_test = x_test / 255 # 把所有 pixel 的值標準化到 0 到 1 之間
labels = model.predict_classes(x_test) # 把 x_test 丟到 predict_classes 就會回傳預測的類別
dataFrame = pandas.DataFrame({"id":ids,"label":labels}) # 把 label 和 id 丟到 DataFrame，方便等一下輸出 csv
dataFrame.to_csv("my_submit.csv",index = False) # 輸出 csv 檔，index = False 的話，csv 檔左邊就不會多一排 index